## Analysis notebook

In this notebook I'll be running the pymc3 model code and trying to interpret results from the posterior. 

Of particular interest is the way $\theta_{4}$ and $\theta_{5}$ behave in the hierarchical context, as the MLE of fully pooled data shows the behaviour you would expect (i.e. increasing probability of long pause with fullness) but preliminary runs of the inference disagree!

First, load the data:

In [5]:
%pwd

u'/home/tmm13/Dropbox/Tom/PhD/projects/PDMP_modelling'

In [6]:
import sys
import os
import theano
import theano.tensor as tt
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import PDMP_ll as ll
import cPickle as pickle

#theano.config.mode = 'FAST_COMPILE'
theano.config.mode = 'FAST_RUN'

run_name = 'new_all_data3'

"""
Data import
"""
min_num_animals = 5

root_data_dir = 'new_all_data/'
group_paths = os.listdir(root_data_dir)
print group_paths

def data_from_file(filename):
    data = np.loadtxt(filename,delimiter='\t',usecols=(0,1,2,3,4))

    f_lengths = data[:,0]
    g_starts = data[:,1]
    rates = data[:,2]
    p_lengths = data[:,3]
    g_ends = data[:,4]

    obs = [f_lengths, g_starts, rates, p_lengths, g_ends]

    return obs

## Iterate over groups
data_holder = []
idx_holder = []
group_idx_holder = []
group_sizes = []
count = 0

subj_data = []

for group_num, path in enumerate(group_paths):
    group_path = root_data_dir+path
    
    if len(os.listdir(group_path)) >= min_num_animals:
        group_sizes.append(len(os.listdir(group_path)))
        print "Using ", group_path
        ## Iterate over members of group
        for filename in os.listdir(group_path):
            if filename [-3:] != 'CSV':
                continue
                
            print filename

            new_data = data_from_file(group_path+'/'+filename)
            data_holder.append(new_data)

            ## Create index holders
            idx_holder.append(count*np.ones(len(new_data[0])))
            group_idx_holder.append(group_num*np.ones(len(new_data[0])))
            
            ## Store subject data
            subj_data.append([group_path, filename])

            count += 1
        
    else:
        print "Skipping ", group_path

data = np.hstack(data_holder)
idx = np.hstack(idx_holder)
group_idx = np.hstack(group_idx_holder)
idx = idx.astype(int)
group_idx = group_idx.astype(int)

num_animals = len(data_holder)
num_groups = np.max(group_idx) + 1

print "Number of data points: ", len(group_idx)
print "Number of animals: ", num_animals
print "Number of groups: ", num_groups

print group_paths

['vehicle_0.0_A_D', 'LiCl_32.0_A_D', 'PYY_7.5_A_D', 'GLP-1_300.0_A_D', 'PYY_300.0_A_D', 'sib_0.9_A_D', 'saline_0.0_R_L', 'LiCl_64.0_R_L', 'PYY_1.5_R_L', 'Ex-4_0.3_A_D', 'PYY_7.5_R_L', 'Lep_2.0_A_D', 'LiCl_32.0_R_L', 'saline_0.0_R_D', 'saline_0.0_A_D', 'GLP-1_100.0_A_D', 'saline_0.0_A_L', 'PYY_1.5_A_D', 'GLP-1_30.0_A_D', 'PYY_300.0_R_L', 'LiCl_16.0_R_L']
Using  new_all_data/vehicle_0.0_A_D
vehicle_0.0_A_D_5_8_2015-03-13.B0105.CSV
vehicle_0.0_A_D_6_8_2015-03-13.B0106.CSV
vehicle_0.0_A_D_4_8_2015-03-13.B0104.CSV
vehicle_0.0_A_D_3_8_2015-03-13.B0103.CSV
vehicle_0.0_A_D_14_8_2015-03-13.B0114.CSV
vehicle_0.0_A_D_15_8_2015-03-13.B0115.CSV
vehicle_0.0_A_D_13_8_2015-03-13.B0113.CSV
vehicle_0.0_A_D_8_8_2015-03-13.B0108.CSV
Using  new_all_data/LiCl_32.0_A_D
LiCl_32.0_A_D_13_8_2015-03-23.B0113.CSV
LiCl_32.0_A_D_3_8_2015-02-17.B0103.CSV
LiCl_32.0_A_D_11_8_2015-02-17.B0111.CSV
LiCl_32.0_A_D_7_8_2015-03-23.B0107.CSV
LiCl_32.0_A_D_5_8_2015-02-17.B0105.CSV
LiCl_32.0_A_D_9_8_2015-02-09.B0109.CSV
LiCl_32

In [ ]:
print data.shape
print group_sizes

for i in subj_data:
    print i[1].split('_')

(5, 6823)
[8, 14, 11, 10, 14, 8, 28, 11, 9, 11, 7, 6, 10, 13, 36, 9, 11, 13, 8, 9, 5]
['vehicle', '0.0', 'A', 'D', '5', '8', '2015-03-13.B0105.CSV']
['vehicle', '0.0', 'A', 'D', '6', '8', '2015-03-13.B0106.CSV']
['vehicle', '0.0', 'A', 'D', '4', '8', '2015-03-13.B0104.CSV']
['vehicle', '0.0', 'A', 'D', '3', '8', '2015-03-13.B0103.CSV']
['vehicle', '0.0', 'A', 'D', '14', '8', '2015-03-13.B0114.CSV']
['vehicle', '0.0', 'A', 'D', '15', '8', '2015-03-13.B0115.CSV']
['vehicle', '0.0', 'A', 'D', '13', '8', '2015-03-13.B0113.CSV']
['vehicle', '0.0', 'A', 'D', '8', '8', '2015-03-13.B0108.CSV']
['LiCl', '32.0', 'A', 'D', '13', '8', '2015-03-23.B0113.CSV']
['LiCl', '32.0', 'A', 'D', '3', '8', '2015-02-17.B0103.CSV']
['LiCl', '32.0', 'A', 'D', '11', '8', '2015-02-17.B0111.CSV']
['LiCl', '32.0', 'A', 'D', '7', '8', '2015-03-23.B0107.CSV']
['LiCl', '32.0', 'A', 'D', '5', '8', '2015-02-17.B0105.CSV']
['LiCl', '32.0', 'A', 'D', '9', '8', '2015-02-09.B0109.CSV']
['LiCl', '32.0', 'A', 'D', '9', '8', '2

## Running the pymc3 model

This code just runs the pymc3 inference as we've done before.

In [ ]:
## Set constants
k1 = 0.00055
num_samples = 5000
nchains = 1
ntune = 5000

## Model setup
with pm.Model() as model:
    ## Group mean
    #means = [-2, -2.35, -2.8, 1, 1, -2, 3.5, 2] # from unpooled data, all vars
    means = [-2, -2, -3, 1, 1, -2, 3, 3] # from unpooled data, all vars
    #means = [-2, -2.35, -2.8] # from unpooled data, theta1 to theta4
    #means = [-1, -2, -2, 3.5, 2] # from unpooled data, theta5 to theta9
    #means = [-1, -1, -1] # test how strongly priors affect posterior
    num_vars = len(means)
    cov = np.eye(num_vars)

    mu = pm.Normal('mu', mu=means, sd=2, shape=(num_groups,num_vars))

    #mu_print = theano.printing.Print('mu')(mu)

    theta_holder = []
    for i in range(0, num_groups):
        ## Create covariance matrix from LKJ
        #sd_dist = pm.HalfCauchy.dist(beta=2.5, shape=num_vars)
        sd_dist = pm.HalfNormal.dist(sd=2, shape=num_vars)
        packed_chol = pm.LKJCholeskyCov('chol_cov'+str(i), eta=1, n=num_vars, sd_dist=sd_dist)
        chol = pm.expand_packed_triangular(num_vars, packed_chol, lower=True)

        theta_tilde = pm.Normal('theta_tilde'+str(i), mu=0, sd=1, shape=(group_sizes[i], num_vars))
        theta_gp = tt.dot(chol, theta_tilde.T).T # have to take transpose to fit with distribution shape
        theta_holder.append(theta_gp)

    thetas = tt.concatenate(theta_holder)

    theta1 = pm.Deterministic('theta1', mu[group_idx,0] + thetas[idx,0]) 
    theta2 = pm.Deterministic('theta2', mu[group_idx,1] + thetas[idx,1]) 
    theta3 = pm.Deterministic('theta3', mu[group_idx,2] + thetas[idx,2]) 
    theta4 = pm.Deterministic('theta4', mu[group_idx,3] + thetas[idx,3]) 
    theta5 = pm.Deterministic('theta5', mu[group_idx,4] + thetas[idx,4]) 
    theta6 = pm.Deterministic('theta6', mu[group_idx,5] + thetas[idx,5]) 
    theta7 = pm.Deterministic('theta7', mu[group_idx,6] + thetas[idx,6]) 
    theta8 = pm.Deterministic('theta8', mu[group_idx,7] + thetas[idx,7])
    
    """
    Power-transform
    """
    p10_theta1 = tt.pow(10., theta1)
    p10_theta2 = tt.pow(10., theta2)
    p10_theta3 = tt.pow(10., theta3)
    p10_theta6 = tt.pow(10., theta6)
    p10_theta7 = tt.pow(10., theta7)
    p10_theta8 = tt.pow(10., theta8)

    ## Likelihood of observations

    ## Exponential feeding bout length
    feeding_lengths = pm.Exponential('f_len', p10_theta1, observed=data[0,:])

    ## Normal feeding bout rate
    rates = pm.Normal('rate', p10_theta2, sd=p10_theta3, observed=data[2,:])

    ## Pause likelihood
    pauses = ll.pause_ll('pause', theta4, theta5, p10_theta6, p10_theta7, p10_theta8, k1, observed=data)

    ## Checking out different step methods to see which works
    # NUTS w/o ADVI - currently fails on LKJCholeskyCov
    trace = pm.sample(num_samples, tune=ntune, njobs=nchains, step=pm.NUTS(), target_accept=0.7, max_treedepth=10)
    #trace = pm.sample(num_samples)

    pm.traceplot(trace)
    plt.savefig('all_data_cov_diag_new.png')
    plt.show()
    
pickle.dump(trace, open(run_name+"_trace.p", "wb"))
pickle.dump(subj_data, open(run_name+"_subj.p", "wb"))
pickle.dump(group_paths, open(run_name+"_paths.p", "wb"))

100%|██████████| 10000/10000 [29:34:31<00:00, 12.00s/it]  /home/tmm13/anaconda2/lib/python2.7/site-packages/pymc3/step_methods/hmc/nuts.py:459: UserWarning: Chain 0 reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
  'reparameterize.' % self._chain_id)
/home/tmm13/anaconda2/lib/python2.7/site-packages/pymc3/step_methods/hmc/nuts.py:451: UserWarning: The acceptance probability in chain 0 does not match the target. It is 0.634968979295, but should be close to 0.8. Try to increase the number of tuning steps.
  % (self._chain_id, mean_accept, target_accept))



In [ ]:
print trace['mu'].shape

In [ ]:
print trace['mu'][::10,0,0].shape

In [ ]:
#trace = pickle.load(open("all_groups_trace.p", "rb"))
num_vars = 8

fig, axes = plt.subplots(num_vars,num_vars,figsize=(15,15))

thin_param = 1
tune_param = 0
alpha = 0.1

#group_list = [7, 13, 15, 3]
group_list = [0,1,2,3,4,5,6,7]
#group_list = np.arange(len(group_paths))

for first_theta in range(0,num_vars):
    print first_theta
    for second_theta in range(0,num_vars):
        #for i in range(0,num_groups):
        for i in group_list:
            #axes[second_theta, first_theta].scatter(np.mean(trace['mu'][tune_param::thin_param,i,first_theta]), np.mean(trace['mu'][tune_param::thin_param,i,second_theta]), alpha=alpha)
            axes[second_theta, first_theta].scatter(trace['mu'][tune_param::thin_param,i,first_theta], trace['mu'][tune_param::thin_param,i,second_theta], alpha=alpha, label=group_paths[i])            

print group_paths

plt.savefig(run_name+'_cov.png')            
plt.show()